## Projet Greedy Algorithm, NAVARRE Victor CACHERA Alexandre

### Modules

In [ ]:
#Importation des librairies

import numpy as np 
import matplotlib.pyplot as plt

## PyMOR

### Données 

In [ ]:
# Premier test
t = np.linspace(0, 1, 100)
K = np.linspace(0, 10, 100)
F1 = [np.sin(k * t) for k in K]

# Deuxième test 

### Algorithmes Gloutons

#### Fonctions de base 

In [ ]:
def proj(f, Vn):

    V = np.column_stack(Vn) 
    coeffs, _, _, _ = np.linalg.lstsq(V, f, rcond=None)
    proj = V @ coeffs
    return proj

from scipy.optimize import minimize

#Si la première méthode ne fonctionne pas :
def proj2(f, V):
    """Projection sur V pour la norme L1 (approchée via optimisation)."""
    A = np.column_stack(V)
    def loss(x):
        return np.linalg.norm(A @ x - f, ord=1)
    res = minimize(loss, np.zeros(len(V)))
    return A @ res.x

def dist(f,Vn):
    return(np.linalg.norm(f-proj(f,Vn)))

#### Algorithme Strong Greedy

In [ ]:
def stronggreedy(F, n):
    
    Vn=[]
    Sigma = []
    id_f0 = np.argmax([np.linalg.norm(f) for f in F]) 
    f0 = F[id_f0]
    Vn.append(f0)           
    Sigma.append(np.linalg.norm(f0))

    for i in range(n):
        id_f = np.argmax([dist(f, Vn) for f in F]) 
        f = F[id_f]
        Sigma.append(dist(f,Vn))
        Vn.append(f)            
    
    return (Vn, Sigma) 

#### Algorithme Weak Greedy

On commence par décrire l'Algorithme Greedy faible pour les n-withd, on réalise une implémentation récursive

In [ ]:
def weakgreedy(A, gamma, n, sample_size):
    
    Vn=[]
    Sigma = []
    id_f0 = np.argmax([np.linalg.norm(f) for f in F]) 
    f0 = F[id_f0]
    Vn.append(f0)          
    Sigma.append(np.linalg.norm(f0))
    
    for i in range(n):
        
        # On cherche une approximation de la distance maximale
        sample_indices = np.random.choice(n, size=sample_size, replace=False)
        sample_distances = []
        
        for idx in sample_indices:
            f = F[idx]
            sample_distances.append(dist(f,Vn))

        current_max = np.max(sample_distances)

        for f in A:   #ici premier choisi, sinon les tirer tous et choisir &léatoirement parmi les bons candidats
            if dist(f,Vn) >= gamma*current_max:
                Sigma.append(dist(f,Vn))
                Vn.append(f)
                break
    return(Vn, Sigma)


### Algorithme de PCA 

### Comparaison et Analyse des résultats

### Conclusion
